### stage_model

prediction models on stage dataset

data format:
    target , eventid ,    car_number,    stageid,     features...

In [28]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# to use only one GPU.
# use this on r-001
# otherwise comment
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb


In [42]:
# bulid regression model
classifiers = ['currank','avgrank','dice','lr','lrl1','lsvc','lsvcl2','rf','lrbias','xgb']
def get_classifier(classifier = 'lr'):
    
    class_weight = None
    
    if classifier == "lsvc":
        clf = LinearSVC(penalty='l1',dual=False, tol=1e-3, class_weight=class_weight )
    elif classifier == "lsvcl2":
        clf = LinearSVC(penalty='l2', tol=1e-4, class_weight=class_weight)
    elif classifier == 'rf':
        #clf = RandomForestClassifier(n_estimators=100, n_jobs=4,criterion='entropy', min_samples_split=1,class_weight = class_weight)
        clf = RandomForestClassifier(n_estimators=100, n_jobs=-1,criterion='entropy', class_weight = class_weight)
    elif classifier == 'lr':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = False, verbose = 0)
    elif classifier == 'lrbias':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = True, verbose = 1)
    elif classifier == 'lrl1':
        clf = LogisticRegression(class_weight = class_weight, penalty='l1',n_jobs=-1)
    elif classifier == 'xgb':
        clf = xgb.XGBClassifier(booster = 'gbtree', nthread = -1, subsample = 1, n_estimators = 600, colsample_bytree = 1, max_depth = 3, min_child_weight = 1)
    elif classifier == 'dice':
        clf = RandomDice('1234')
    elif classifier == 'currank':
        clf = CurRank()
    elif classifier == 'avgrank':
        clf = AverageRank()        
    else:
        clf = None
        
    return clf


class CurRank():
    """
    predict with current rank
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = [0 for x in range(test_x.shape[0])]
        return np.array(pred_y)
    
class AverageRank():
    """
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            #13, change_in_rank_all
            pred_y.append(x[13])
        pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in pred_y])
        return np.array(pred_y_avg)   

class RandomDice():
    """
    a random dice model
    """
    def __init__(self, seed='1234'):
        self.dist = []
        self.val = []
        random.seed(seed)
    
    def fit(self, x, y):
        total = y.shape[0]
        yval = set(y)
        
        ratio = 0.
        for val in yval:
            self.val.append(val)
            ratio += np.sum(y==val)*1.0 / total
            self.dist.append(ratio)
            
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            dice = random.random()
            #search in self.dist
            find_idx = -1
            for idx, ratio in enumerate(self.dist):
                if dice <= ratio:
                    find_idx = idx
                    break
            
            #or the last one match
            pred_y.append(self.val[find_idx])
            
        return np.array(pred_y)

def evaluate(test_y, pred_y):
    precision = metrics.precision_score(test_y, pred_y, average=None) 
    recall = metrics.recall_score(test_y, pred_y, average=None)
    f1 = metrics.f1_score(test_y, pred_y, average=None)
    accuracy = metrics.accuracy_score(test_y, pred_y)
    print('precision=%s, recall=%s, f1=%s, accuracy=%.2f'%(precision,recall, f1, accuracy))
    return accuracy
    
#
#features
#    cols=[Myidx, 'target','eventid','car_number','stageid',
#             'firststage','pit_in_caution','start_position',
#             'start_rank','start_rank_ratio','top_pack','bottom_pack',
#             'average_rank','average_rank_all',
#             'change_in_rank','change_in_rank_all','rate_of_change','rate_of_change_all']    
def split_by_eventid(stagedata, eventid):
    """
    split by eventid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['eventid'] != eventid].to_numpy()
    test  = stagedata[stagedata['eventid'] == eventid].to_numpy()

    #2:car_number
    train_x = train[:,2:]
    train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    test_x = test[:,2:]
    test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    
    return train, test, train_x, train_y, test_x, test_y


def split_by_stageid(stagedata, stageid):
    """
    split by stageid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['stageid'] <= stageid].to_numpy()
    test  = stagedata[stagedata['stageid'] > stageid].to_numpy()

    train_x = train[:,2:]
    train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    test_x = test[:,2:]
    test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    
    return train, test, train_x, train_y, test_x, test_y


### baseline
def baseline_model():
    #1. predict with current rank, rankchg = 0
    print('[*] predict with current rank, rankchg = 0')
    pred_y_simple = np.zeros_like(test_y)
    score1 = evaluate(test_y, pred_y_simple)

    #2. predict with average rankchg (change_in_rank_all):idx = 15
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    score2 = evaluate(test_y, pred_y_avg)
    return score1, score2

def classifier_model(name='lr'):
    ### test learning models
    print('[*] predict with %s model'%name)
    clf = get_classifier(name)
    clf.fit(train_x, train_y)

    pred_y = clf.predict(test_x)
    score = evaluate(test_y, pred_y)
    return score

In [8]:
#load data
stagedata = pd.read_csv('stage-2018.csv')
stagedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805 entries, 0 to 804
Data columns (total 18 columns):
Unnamed: 0            805 non-null int64
target                805 non-null int64
eventid               805 non-null int64
car_number            805 non-null int64
stageid               805 non-null int64
firststage            805 non-null int64
pit_in_caution        805 non-null int64
start_position        805 non-null int64
start_rank            805 non-null int64
start_rank_ratio      805 non-null float64
top_pack              805 non-null int64
bottom_pack           805 non-null int64
average_rank          805 non-null float64
average_rank_all      805 non-null float64
change_in_rank        805 non-null int64
change_in_rank_all    805 non-null float64
rate_of_change        805 non-null int64
rate_of_change_all    805 non-null float64
dtypes: float64(5), int64(13)
memory usage: 113.3 KB


In [ ]:
stagedata

In [10]:
eventsname = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
events = set(stagedata['eventid'])
for eventid in events:
    print('Testset = %s'%eventsname[eventid])
    
    train, test, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, eventid)
    
    score1, score2 = baseline_model()
    
    score3 = classifier_model()

    

Testset = Phoenix
[*] predict with current rank, rankchg = 0
precision=[0.         0.14035088 0.        ], recall=[0. 1. 0.], accuracy=0.14
[*] predict with average rankchg (change_in_rank_all):idx = 15
precision=[0.48275862 0.16129032 0.16      ], recall=[0.46666667 0.3125     0.10526316], accuracy=0.32
[*] predict with lr model
precision=[0.625      0.27272727 0.4516129 ], recall=[0.75       0.1875     0.36842105], accuracy=0.54
Testset = Indy500
[*] predict with current rank, rankchg = 0
precision=[0.         0.20888889 0.        ], recall=[0. 1. 0.], accuracy=0.21
[*] predict with average rankchg (change_in_rank_all):idx = 15
precision=[0.4017094  0.24528302 0.23636364], recall=[0.48958333 0.27659574 0.15853659], accuracy=0.32
[*] predict with lr model
precision=[0.56834532 0.33333333 0.58441558], recall=[0.82291667 0.06382979 0.54878049], accuracy=0.56
Testset = Texas
[*] predict with current rank, rankchg = 0
precision=[0.         0.26771654 0.        ], recall=[0. 1. 0.], accura

/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 48.
  " = {}.".format(effective_n_j

In [16]:
for stageid in range(8):
    train, test, train_x, train_y, test_x, test_y =split_by_stageid(stagedata, stageid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
          (stageid, train_x.shape[0], test_x.shape[0], test_distribution))
    
    baseline_model()
    
    classifier_model()
    

Testset by stageid= 0, trainsize=153, testsize=652, dist=+:221,0:167,-:264
[*] predict with current rank, rankchg = 0
precision=[0.         0.25613497 0.        ], recall=[0. 1. 0.], accuracy=0.26
[*] predict with average rankchg (change_in_rank_all):idx = 15
precision=[0.37172775 0.22535211 0.23115578], recall=[0.53787879 0.09580838 0.2081448 ], accuracy=0.31
[*] predict with lr model
precision=[0.49633252 0.36842105 0.56603774], recall=[0.76893939 0.41916168 0.13574661], accuracy=0.46
Testset by stageid= 1, trainsize=288, testsize=517, dist=+:186,0:136,-:195
[*] predict with current rank, rankchg = 0
precision=[0.         0.26305609 0.        ], recall=[0. 1. 0.], accuracy=0.26
[*] predict with average rankchg (change_in_rank_all):idx = 15
precision=[0.34674923 0.24242424 0.24223602], recall=[0.57435897 0.05882353 0.20967742], accuracy=0.31
[*] predict with lr model
precision=[0.42746114 0.44642857 0.58666667], recall=[0.84615385 0.18382353 0.23655914], accuracy=0.45
Testset by stage

/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 48.
  " = {}.".format(effective_n_j

In [48]:
cols = ['runid','trainsize','testsize','testdistribution']
cols.extend(classifiers)
print('cols:%s'%cols)
retdf = pd.DataFrame([],columns=cols)

eventsname = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
events = set(stagedata['eventid'])
for eventid in events:
    print('Testset = %s'%eventsname[eventid])
    
    train, test, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, eventid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
          (stageid, train_x.shape[0], test_x.shape[0], test_distribution))
    
    #record
    rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]
    
    acc = [0 for x in range(len(classifiers))]
    for idx, clf in enumerate(classifiers):
        acc[idx] = classifier_model(clf)

    rec.extend(acc)
    print('rec:%s'%rec)
    
    #new df
    df = pd.DataFrame([rec],columns=cols)
    retdf = pd.concat([retdf, df])        
    

df_event = retdf

cols:['runid', 'trainsize', 'testsize', 'testdistribution', 'currank', 'avgrank', 'dice', 'lr', 'lrl1', 'lsvc', 'lsvcl2', 'rf', 'lrbias', 'xgb']
Testset = Phoenix
Testset by stageid= 7, trainsize=691, testsize=114, dist=+:38,0:16,-:60
[*] predict with currank model
precision=[0.         0.14035088 0.        ], recall=[0. 1. 0.], f1=[0.         0.24615385 0.        ], accuracy=0.14
[*] predict with avgrank model
precision=[0.48275862 0.16129032 0.16      ], recall=[0.46666667 0.3125     0.10526316], f1=[0.47457627 0.21276596 0.12698413], accuracy=0.32
[*] predict with dice model
precision=[0.52173913 0.2        0.31578947], recall=[0.4        0.375      0.31578947], f1=[0.45283019 0.26086957 0.31578947], accuracy=0.37
[*] predict with lr model
precision=[0.625      0.27272727 0.4516129 ], recall=[0.75       0.1875     0.36842105], f1=[0.68181818 0.22222222 0.4057971 ], accuracy=0.54
[*] predict with lrl1 model
precision=[0.62666667 0.27272727 0.46428571], recall=[0.78333333 0.1875     0

/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", 

precision=[0.62686567 0.25       0.42857143], recall=[0.7        0.1875     0.39473684], f1=[0.66141732 0.21428571 0.4109589 ], accuracy=0.53
[*] predict with lsvcl2 model
precision=[0.85714286 0.21428571 0.39240506], recall=[0.3        0.1875     0.81578947], f1=[0.44444444 0.2        0.52991453], accuracy=0.46
[*] predict with rf model
precision=[0.65151515 0.33333333 0.56666667], recall=[0.71666667 0.375      0.44736842], f1=[0.68253968 0.35294118 0.5       ], accuracy=0.58
[*] predict with lrbias model
[LibLinear]precision=[0.63768116 0.25       0.45454545], recall=[0.73333333 0.1875     0.39473684], f1=[0.68217054 0.21428571 0.42253521], accuracy=0.54
[*] predict with xgb model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 48.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


precision=[0.58333333 0.21875    0.54545455], recall=[0.58333333 0.4375     0.31578947], f1=[0.58333333 0.29166667 0.4       ], accuracy=0.47
rec:['Phoenix', 691, 114, '+:38,0:16,-:60', 0.14035087719298245, 0.32456140350877194, 0.3684210526315789, 0.543859649122807, 0.5526315789473685, 0.5263157894736842, 0.45614035087719296, 0.5789473684210527, 0.543859649122807, 0.47368421052631576]
Testset = Indy500
Testset by stageid= 7, trainsize=580, testsize=225, dist=+:82,0:47,-:96
[*] predict with currank model
precision=[0.         0.20888889 0.        ], recall=[0. 1. 0.], f1=[0.         0.34558824 0.        ], accuracy=0.21
[*] predict with avgrank model
precision=[0.4017094  0.24528302 0.23636364], recall=[0.48958333 0.27659574 0.15853659], f1=[0.44131455 0.26       0.18978102], accuracy=0.32
[*] predict with dice model
precision=[0.47619048 0.26315789 0.38095238], recall=[0.52083333 0.31914894 0.29268293], f1=[0.49751244 0.28846154 0.33103448], accuracy=0.40
[*] predict with lr model
prec

...

precision=[0.41304348 0.63157895 0.27868852], recall=[0.52777778 0.19672131 0.5862069 ], f1=[0.46341463 0.3        0.37777778], accuracy=0.38
rec:['Pocono', 679, 126, '+:29,0:61,-:36', 0.48412698412698413, 0.2777777777777778, 0.23809523809523808, 0.3412698412698413, 0.3253968253968254, 0.3253968253968254, 0.49206349206349204, 0.3333333333333333, 0.3333333333333333, 0.38095238095238093]
Testset = Gateway
Testset by stageid= 7, trainsize=701, testsize=104, dist=+:34,0:28,-:42
[*] predict with currank model
precision=[0.         0.26923077 0.        ], recall=[0. 1. 0.], f1=[0.         0.42424242 0.        ], accuracy=0.27
[*] predict with avgrank model
precision=[0.36956522 0.29411765 0.29166667], recall=[0.4047619  0.35714286 0.20588235], f1=[0.38636364 0.32258065 0.24137931], accuracy=0.33
[*] predict with dice model
precision=[0.34782609 0.13043478 0.2       ], recall=[0.38095238 0.10714286 0.20588235], f1=[0.36363636 0.11764706 0.20289855], accuracy=0.25
[*] predict with lr model
pre

/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", 

precision=[0.90909091 0.33333333 0.5       ], recall=[0.23809524 0.96428571 0.17647059], f1=[0.37735849 0.49541284 0.26086957], accuracy=0.41
[*] predict with lsvcl2 model
precision=[0.51515152 0.33333333 0.6       ], recall=[0.80952381 0.39285714 0.08823529], f1=[0.62962963 0.36065574 0.15384615], accuracy=0.46
[*] predict with rf model
precision=[0.52941176 0.30769231 0.32258065], recall=[0.42857143 0.42857143 0.29411765], f1=[0.47368421 0.35820896 0.30769231], accuracy=0.38
[*] predict with lrbias model
[LibLinear]precision=[0.83333333 0.325      0.5       ], recall=[0.23809524 0.92857143 0.17647059], f1=[0.37037037 0.48148148 0.26086957], accuracy=0.40
[*] predict with xgb model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 48.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


precision=[0.55263158 0.29268293 0.36      ], recall=[0.5        0.42857143 0.26470588], f1=[0.525      0.34782609 0.30508475], accuracy=0.40
rec:['Gateway', 701, 104, '+:34,0:28,-:42', 0.2692307692307692, 0.3269230769230769, 0.25, 0.4519230769230769, 0.41346153846153844, 0.41346153846153844, 0.46153846153846156, 0.38461538461538464, 0.40384615384615385, 0.40384615384615385]


In [51]:
retdf.to_csv('crossvalid_stagedata_splitbyevent.csv')

In [50]:
retdf

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lrl1,lsvc,lsvcl2,rf,lrbias,xgb
0,Phoenix,691,114,"+:38,0:16,-:60",0.140351,0.324561,0.368421,0.543860,0.552632,0.526316,0.456140,0.578947,0.543860,0.473684
0,Indy500,580,225,"+:82,0:47,-:96",0.208889,0.324444,0.395556,0.564444,0.555556,0.542222,0.546667,0.533333,0.555556,0.520000
0,Texas,678,127,"+:39,0:34,-:54",0.267717,0.314961,0.291339,0.598425,0.606299,0.622047,0.543307,0.496063,0.606299,0.433071
0,Iowa,696,109,"+:42,0:28,-:39",0.256881,0.293578,0.256881,0.357798,0.348624,0.339450,0.302752,0.321101,0.348624,0.293578
0,Pocono,679,126,"+:29,0:61,-:36",0.484127,0.277778,0.238095,0.341270,0.325397,0.325397,0.492063,0.333333,0.333333,0.380952
0,Gateway,701,104,"+:34,0:28,-:42",0.269231,0.326923,0.250000,0.451923,0.413462,0.413462,0.461538,0.384615,0.403846,0.403846


In [52]:
df_event = retdf

In [53]:
retdf = pd.DataFrame([],columns=cols)

for stageid in range(8):
    train, test, train_x, train_y, test_x, test_y =split_by_stageid(stagedata, stageid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
    #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))
    
    #record
    rec = ['stage%d'%stageid,train_x.shape[0],test_x.shape[0],test_distribution]
    
    acc = [0 for x in range(len(classifiers))]
    for idx, clf in enumerate(classifiers):
        acc[idx] = classifier_model(clf)

    rec.extend(acc)
    print('rec:%s'%rec)
    
    #new df
    df = pd.DataFrame([rec],columns=cols)
    retdf = pd.concat([retdf, df])  
    
retdf.to_csv('crossvalid_stagedata_splitbystage.csv')
df_stage = retdf

[*] predict with currank model
precision=[0.         0.25613497 0.        ], recall=[0. 1. 0.], f1=[0.         0.40781441 0.        ], accuracy=0.26
[*] predict with avgrank model
precision=[0.37172775 0.22535211 0.23115578], recall=[0.53787879 0.09580838 0.2081448 ], f1=[0.43962848 0.13445378 0.21904762], accuracy=0.31
[*] predict with dice model
precision=[0.41197183 0.25806452 0.3956044 ], recall=[0.44318182 0.28742515 0.32579186], f1=[0.4270073  0.27195467 0.3573201 ], accuracy=0.36
[*] predict with lr model
precision=[0.49633252 0.36842105 0.56603774], recall=[0.76893939 0.41916168 0.13574661], f1=[0.60326895 0.39215686 0.2189781 ], accuracy=0.46
[*] predict with lrl1 model
precision=[0.50534759 0.36190476 0.48529412], recall=[0.71590909 0.45508982 0.14932127], f1=[0.59247649 0.40318302 0.2283737 ], accuracy=0.46
[*] predict with lsvc model
precision=[0.47645429 0.3375     0.17647059], recall=[0.65151515 0.48502994 0.04072398], f1=[0.5504     0.3980344  0.06617647], accuracy=0.40


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", 

precision=[0.44444444 0.33333333 0.3566879 ], recall=[0.5        0.39520958 0.25339367], f1=[0.47058824 0.36164384 0.2962963 ], accuracy=0.39
[*] predict with lrbias model
[LibLinear]precision=[0.49234694 0.35576923 0.48076923], recall=[0.73106061 0.44311377 0.11312217], f1=[0.58841463 0.39466667 0.18315018], accuracy=0.45
[*] predict with xgb model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 48.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


precision=[0.4497992  0.29338843 0.34782609], recall=[0.42424242 0.4251497  0.25339367], f1=[0.43664717 0.34718826 0.29319372], accuracy=0.37
rec:['stage0', 153, 652, '+:221,0:167,-:264', 0.2561349693251534, 0.3128834355828221, 0.36349693251533743, 0.4647239263803681, 0.4570552147239264, 0.401840490797546, 0.455521472392638, 0.3895705521472393, 0.44785276073619634, 0.3665644171779141]
[*] predict with currank model
precision=[0.         0.26305609 0.        ], recall=[0. 1. 0.], f1=[0.         0.41653905 0.        ], accuracy=0.26
[*] predict with avgrank model
precision=[0.34674923 0.24242424 0.24223602], recall=[0.57435897 0.05882353 0.20967742], f1=[0.43243243 0.09467456 0.22478386], accuracy=0.31
[*] predict with dice model
precision=[0.39312977 0.22881356 0.37956204], recall=[0.52820513 0.19852941 0.27956989], f1=[0.45076586 0.21259843 0.32198142], accuracy=0.35
[*] predict with lr model
precision=[0.42746114 0.44642857 0.58666667], recall=[0.84615385 0.18382353 0.23655914], f1=[0

...

precision=[1.   0.25 0.5 ], recall=[0.25 0.75 0.25], f1=[0.4        0.375      0.33333333], accuracy=0.38
[*] predict with lsvc model
precision=[0.66666667 0.25       1.        ], recall=[0.25 0.75 0.25], f1=[0.36363636 0.375      0.4       ], accuracy=0.38
[*] predict with lsvcl2 model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


precision=[0.   0.25 0.  ], recall=[0. 1. 0.], f1=[0.  0.4 0. ], accuracy=0.25
[*] predict with rf model
precision=[0.55555556 0.         0.75      ], recall=[0.625 0.    0.75 ], f1=[0.58823529 0.         0.75      ], accuracy=0.50
[*] predict with lrbias model
[LibLinear]precision=[1.   0.25 0.5 ], recall=[0.25 0.75 0.25], f1=[0.4        0.375      0.33333333], accuracy=0.38
[*] predict with xgb model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 48.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


precision=[0.5        0.2        0.42857143], recall=[0.25 0.25 0.75], f1=[0.33333333 0.22222222 0.54545455], accuracy=0.38
rec:['stage7', 789, 16, '+:4,0:4,-:8', 0.25, 0.1875, 0.3125, 0.375, 0.375, 0.375, 0.25, 0.5, 0.375, 0.375]


In [54]:
df_stage

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lrl1,lsvc,lsvcl2,rf,lrbias,xgb
0,stage0,153,652,"+:221,0:167,-:264",0.256135,0.312883,0.363497,0.464724,0.457055,0.401840,0.455521,0.389571,0.447853,0.366564
0,stage1,288,517,"+:186,0:136,-:195",0.263056,0.307544,0.352031,0.452611,0.483559,0.439072,0.390716,0.448743,0.454545,0.462282
0,stage2,421,384,"+:140,0:112,-:132",0.291667,0.263021,0.317708,0.484375,0.476562,0.471354,0.390625,0.455729,0.473958,0.427083
0,stage3,547,258,"+:91,0:89,-:78",0.344961,0.228682,0.333333,0.484496,0.476744,0.480620,0.476744,0.430233,0.472868,0.437984
0,stage4,657,148,"+:48,0:53,-:47",0.358108,0.209459,0.344595,0.554054,0.560811,0.554054,0.432432,0.513514,0.574324,0.459459
0,stage5,725,80,"+:26,0:29,-:25",0.362500,0.200000,0.325000,0.575000,0.587500,0.537500,0.325000,0.575000,0.575000,0.487500
0,stage6,767,38,"+:11,0:13,-:14",0.342105,0.236842,0.263158,0.526316,0.526316,0.526316,0.368421,0.552632,0.526316,0.605263
0,stage7,789,16,"+:4,0:4,-:8",0.250000,0.187500,0.312500,0.375000,0.375000,0.375000,0.250000,0.500000,0.375000,0.375000
